In [1]:
# given vine structure, copulas and copula parameters
# fix var5 as missing
# for all 1...n, impute 5 using 1,...,4

# step1. given (a) vine structure, (b) copula family and  (c) copula parameters
# step2. generate data
# step3. mask var5 with x% dropout
# step4. infer the structure using complete cases
# step5.1 if the vine structure supports direct rimputation of 5 -> easy
# step5.2 if the vine structure does not support direct imputation of 5 -> harder ?
# step6. run zhao/udell's code, conditional mean imputation
# step7. smae imputation error, bias of correlation

# the pair-copula index for edge e in tree t of a d dimensional vine is
# (M[d - 1 - e, e], M[t, e]; M[t - 1, e], ..., M[0, e])

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pyvinecopulib as pv

In [3]:
d = 5
corr = np.array([
    [0.5, 0.25, 0.25, 0.9], 
    [0.1, 0.9, 0.9, np.nan], 
    [0.125, 0.66, np.nan, np.nan],
    [0.55, np.nan, np.nan, np.nan]]
)
vine_matrix = np.array([
    [3,2,3,3,3],
    [2,3,2,2,0],
    [4,4,4,0,0],
    [1,1,0,0,0],
    [5,0,0,0,0]
])

In [4]:
pair_copulas = []
for t in range(d-1):
    cur = []
    pair_copulas.append(cur)
    for e in range(d-1-t):
        cur.append(
            pv.Bicop(
                family=pv.BicopFamily.gaussian,
                parameters=[corr[t,e]]
            )
        )
cop = pv.Vinecop(matrix=vine_matrix, pair_copulas=pair_copulas)
u = cop.simulate(1000, seeds=[1,2,3,4,5])

In [5]:
missing = np.random.binomial(n=1, p=0.5, size=u.shape[0])
present = (1-missing)
u_mask = np.copy(u)
u_mask[:, 4] = np.where(dropout > 0, np.nan, u_mask[:,4])

NameError: name 'dropout' is not defined

In [ ]:
fit_controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])
cop = pv.Vinecop(u_mask, controls=fit_controls)

In [ ]:
u1 = u_mask[:, 0][:, None]
u2 = u_mask[:, 1][:, None]
u3 = u_mask[:, 2][:, None]
u4 = u_mask[:, 3][:, None]
u5_true = u[:, 4][:, None]
v = np.random.uniform(size=u_mask.shape[0])[:, None]